In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from extract.polocm import AP, Hypothesis, POLOCM
from traces import *
from observation import PartialOrderedActionObservation
from extract.model import *
import pulp as pl

In [3]:
def get_bw_PO_traces(print_trace=False):
    objects = {
        "A": PlanningObject("unknown", "A"),
        "B": PlanningObject("unknown", "B"),
        "C": PlanningObject("unknown", "C"),
    }

    actions = {
        "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
        "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
        "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
        "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
        "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
        "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
        "stackAB": Action("stack", [objects["A"], objects["B"]]),
        "stackBA": Action("stack", [objects["B"], objects["A"]]),
        "stackAC": Action("stack", [objects["A"], objects["C"]]),
        "stackCA": Action("stack", [objects["C"], objects["A"]]),
        "stackBC": Action("stack", [objects["B"], objects["C"]]),
        "stackCB": Action("stack", [objects["C"], objects["B"]]),
        "putdownA": Action("putdown", [objects["A"]]),
        "putdownB": Action("putdown", [objects["B"]]),
        "putdownC": Action("putdown", [objects["C"]]),
        "pickupA": Action("pickup", [objects["A"]]),
        "pickupB": Action("pickup", [objects["B"]]),
        "pickupC": Action("pickup", [objects["C"]]),
    }

    input_action_seqs = '''
        unstack(A, B), putdown(A), pickup(B), stack(B,C), unstack(B,C), stack(B,C),pickup(A), stack(A,B)
        unstack(C, B), putdown(C), unstack(B,A), putdown(B), pickup(B), stack(B,C), pickup(A), stack(A,B)
        unstack(C,B), stack(C,A), pickup(B), putdown(B), pickup(B), stack(B,C)
        '''
       
    traces = TraceList(
        [   
            PartialOrderedTrace(
                [
                    PartialOrderedStep(State({}), actions["unstackAB"], 1, [2,8]),
                    PartialOrderedStep(State({}), actions["putdownA"], 2, [7]),
                    PartialOrderedStep(State({}), actions["pickupB"], 3, []),
                    PartialOrderedStep(State({}), actions["stackBC"], 4, [5,8]),
                    PartialOrderedStep(State({}), actions["unstackBC"], 5, [6]),
                    PartialOrderedStep(State({}), actions["stackBC"], 6, [7]),
                    PartialOrderedStep(State({}), actions["pickupA"], 7, [8]),
                    PartialOrderedStep(State({}), actions["stackAB"], 8, []),
                ]
            ),
            PartialOrderedTrace(
                [
                    PartialOrderedStep(State({}),actions['unstackCB'], 1, [4,8]),
                    PartialOrderedStep(State({}),actions['putdownC'], 2, [3,5]),
                    PartialOrderedStep(State({}),actions['unstackBA'], 3, [6]),
                    PartialOrderedStep(State({}),actions['putdownB'], 4, [5]),
                    PartialOrderedStep(State({}),actions['pickupB'], 5, [7]),
                    PartialOrderedStep(State({}),actions['stackBC'], 6, [8]),
                    PartialOrderedStep(State({}),actions['pickupA'], 7, []),
                    PartialOrderedStep(State({}),actions['stackAB'], 8, []),
                ]
            ),
            PartialOrderedTrace(
                [
                    PartialOrderedStep(State({}), actions["unstackCB"], 1, [2,3,4,5,6]),
                    PartialOrderedStep(State({}), actions["stackCA"], 2, [3,6]),
                    PartialOrderedStep(State({}), actions["pickupB"], 3, [4]),
                    PartialOrderedStep(State({}), actions["putdownB"], 4, [5,6]),
                    PartialOrderedStep(State({}), actions["pickupB"], 5, []),
                    PartialOrderedStep(State({}), actions["stackBC"], 6, []),
                ]
            ),
            
        ]
    )
    

    if print_trace:
        # traces.print()
        traces.print("details")

    obs = traces.tokenize(PartialOrderedActionObservation)
    return obs

In [4]:
obs_tracelist = get_bw_PO_traces(True)

                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown A unknown B                                           
                        2                    putdown unknown A                                                     
                        3                    pickup unknown B                                                      
                        4                    stack unknown B unknown C                                             
                        5                    unstack unknown B unknown C                                           
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  8 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    putdown unknown C                                                     
                        3                    unstack unknown B unknown A                                           
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C                                             
                        7                    pickup unknown A                                                      
                        8                    stack unknown A unknown B

                                                       Trace                                                       
Attributes:                                                                                                        
  6 steps                                                                                                          
  0 fluents                                                                                                        
                                                       Steps                                                       
                     Step       State        Action                                                                
                        1                    unstack unknown C unknown B                                           
                        2                    stack unknown C unknown A                                             
                        3                    pickup unknown B                                                      
                        4                    putdown unknown B                                                     
                        5                    pickup unknown B                                                      
                        6                    stack unknown B unknown C

In [5]:
sorts = POLOCM._get_sorts(obs_tracelist)
print(f"Sorts:\n{sorts}", end="\n\n")

Sorts:
{'A': 1, 'B': 1, 'C': 1}



In [6]:
obj_PO_trace_overall, trace_PO_matrix_overall, obj_trace_PO_matrix_overall, obj_trace_FO_matrix_overall, sort_aps = POLOCM.polocm_step1(obs_tracelist, sorts, True)

# Step 1

## Obj PO traces:


### Trace **0**


+--------------+--------------+--------------+
| unknown A    | unknown B    | unknown C    |
|--------------+--------------+--------------|
| [1]unstack.1 | [1]unstack.2 | [4]stack.2   |
| [2]putdown.1 | [3]pickup.1  | [5]unstack.2 |
| [7]pickup.1  | [4]stack.1   | [6]stack.2   |
| [8]stack.1   | [5]unstack.1 |              |
|              | [6]stack.1   |              |
|              | [8]stack.2   |              |
+--------------+--------------+--------------+


### Trace **1**


...

╒══════════════╤════════════════╤══════════════╤══════════════╕
│              │   [1]unstack.1 │   [2]stack.1 │   [6]stack.2 │
╞══════════════╪════════════════╪══════════════╪══════════════╡
│ [1]unstack.1 │            nan │            1 │            0 │
├──────────────┼────────────────┼──────────────┼──────────────┤
│ [2]stack.1   │            nan │          nan │            1 │
├──────────────┼────────────────┼──────────────┼──────────────┤
│ [6]stack.2   │            nan │          nan │          nan │
╘══════════════╧════════════════╧══════════════╧══════════════╛


#### Object B


╒══════════════╤════════════════╤═══════════════╤════════════════╤═══════════════╤══════════════╕
│              │   [1]unstack.2 │   [3]pickup.1 │   [4]putdown.1 │   [5]pickup.1 │   [6]stack.1 │
╞══════════════╪════════════════╪═══════════════╪════════════════╪═══════════════╪══════════════╡
│ [1]unstack.2 │            nan │             1 │              0 │             0 │            0 │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼──────────────┤
│ [3]pickup.1  │            nan │           nan │              1 │             0 │            0 │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼──────────────┤
│ [4]putdown.1 │            nan │           nan │            nan │           nan │          nan │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼──────────────┤
│ [5]pickup.1  │            nan │           nan │            nan │           nan │          nan │
├──────────────┼────

#### Object A


╒════════════╤══════════════╕
│            │   [2]stack.2 │
╞════════════╪══════════════╡
│ [2]stack.2 │          nan │
╘════════════╧══════════════╛


In [7]:
prob, PO_vars_overall, PO_matrix_with_vars = POLOCM.polocm_step2(trace_PO_matrix_overall, obj_trace_PO_matrix_overall, debug=True)

# Step 2

## PO vars

### Trace 0

{([1]unstack, [5]unstack): trace_0__1_unstack~_5_unstack,
 ([1]unstack, [4]stack): trace_0__1_unstack~_4_stack,
 ([2]putdown, [3]pickup): trace_0__2_putdown~_3_pickup,
 ([1]unstack, [6]stack): trace_0__1_unstack~_6_stack,
 ([2]putdown, [4]stack): trace_0__2_putdown~_4_stack,
 ([1]unstack, [3]pickup): trace_0__1_unstack~_3_pickup,
 ([2]putdown, [5]unstack): trace_0__2_putdown~_5_unstack,
 ([2]putdown, [6]stack): trace_0__2_putdown~_6_stack,
 ([3]pickup, [6]stack): trace_0__3_pickup~_6_stack,
 ([3]pickup, [7]pickup): trace_0__3_pickup~_7_pickup,
 ([3]pickup, [8]stack): trace_0__3_pickup~_8_stack,
 ([3]pickup, [5]unstack): trace_0__3_pickup~_5_unstack,
 ([3]pickup, [4]stack): trace_0__3_pickup~_4_stack}


### Trace 1

...

╒══════════════╤════════════════╤═══════════════╤════════════════╤═══════════════╤═══════════════════════╕
│              │   [1]unstack.2 │   [3]pickup.1 │   [4]putdown.1 │   [5]pickup.1 │ [6]stack.1            │
╞══════════════╪════════════════╪═══════════════╪════════════════╪═══════════════╪═══════════════════════╡
│ [1]unstack.2 │            nan │             1 │              1 │             1 │ 1                     │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼───────────────────────┤
│ [3]pickup.1  │            nan │           nan │              1 │             1 │ 1                     │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼───────────────────────┤
│ [4]putdown.1 │            nan │           nan │            nan │             1 │ 1                     │
├──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼───────────────────────┤
│ [5]pickup.1  │            nan │    

#### Obj A

╒════════════╤══════════════╕
│            │   [2]stack.2 │
╞════════════╪══════════════╡
│ [2]stack.2 │          nan │
╘════════════╧══════════════╛


## PO Constraints

╒══════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════╕
│ Trace 0                                                                                          │ Trace 1                                                                │ Trace 2                                 │
╞══════════════════════════════════════════════════════════════════════════════════════════════════╪════════════════════════════════════════════════════════════════════════╪═════════════════════════════════════════╡
│ trace_0__1_unstack~_4_stack >= trace_0__1_unstack~_3_pickup + trace_0__3_pickup~_4_stack - 1     │ trace_1__1_unstack~_6_stack >= trace_1__1_unstack~_2_putdown + 1 - 1   │ 1 >= 1 + trace_2__5_pickup~_6_stack - 1 │
├──────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────

In [8]:
prob, FO_vars_overall, FO_matrix_with_vars, varname_lookup = POLOCM.polocm_step3(prob,PO_vars_overall, PO_matrix_with_vars, obj_trace_FO_matrix_overall, debug=True)

# Step 3

## FO vars

### Trace 0

╒══════════════════════════════╤═════════════╤═════════════╕
│ unknown B                    │ unknown A   │ unknown C   │
╞══════════════════════════════╪═════════════╪═════════════╡
│ ([1]unstack.2, [3]pickup.1)  │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([1]unstack.2, [4]stack.1)   │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([1]unstack.2, [5]unstack.1) │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([1]unstack.2, [6]stack.1)   │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([1]unstack.2, [8]stack.2)   │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([3]pickup.1, [4]stack.1)    │             │             │
├──────────────────────────────┼─────────────┼─────────────┤
│ ([3]pickup.1, [5]unstack.1)  │             │             │
├───────────────────────

### Trace 1

...

╒══════════════╤════════════════╤═══════════════╤════════════════╤═════════════════════════════╤════════════════════════════╕
│              │   [1]unstack.2 │   [3]pickup.1 │   [4]putdown.1 │ [5]pickup.1                 │ [6]stack.1                 │
╞══════════════╪════════════════╪═══════════════╪════════════════╪═════════════════════════════╪════════════════════════════╡
│ [1]unstack.2 │            nan │             1 │              0 │ 0                           │ 0                          │
├──────────────┼────────────────┼───────────────┼────────────────┼─────────────────────────────┼────────────────────────────┤
│ [3]pickup.1  │            nan │           nan │              1 │ 0                           │ 0                          │
├──────────────┼────────────────┼───────────────┼────────────────┼─────────────────────────────┼────────────────────────────┤
│ [4]putdown.1 │            nan │           nan │            nan │ ([4]putdown.1, [5]pickup.1) │ ([4]putdown.1, [6]sta

#### Obj A

╒════════════╤══════════════╕
│            │   [2]stack.2 │
╞════════════╪══════════════╡
│ [2]stack.2 │          nan │
╘════════════╧══════════════╛


## FO Constraints

╒═══════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════╤════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════

In [9]:
prob, sort_transition_matrix, sort_AP_vars, varname_lookup = POLOCM.polocm_step4(prob, sort_aps, sorts,FO_vars_overall, FO_matrix_with_vars,varname_lookup, debug=True)

# Step 4

## AP vars

╒════════════════════════╕
│ 1                      │
╞════════════════════════╡
│ (unstack.1, stack.2)   │
├────────────────────────┤
│ (unstack.1, pickup.1)  │
├────────────────────────┤
│ (stack.2, stack.2)     │
├────────────────────────┤
│ (unstack.2, unstack.1) │
├────────────────────────┤
│ (unstack.2, stack.1)   │
├────────────────────────┤
│ (unstack.2, putdown.1) │
├────────────────────────┤
│ (pickup.1, unstack.1)  │
├────────────────────────┤
│ (pickup.1, stack.2)    │
├────────────────────────┤
│ (pickup.1, pickup.1)   │
├────────────────────────┤
│ (stack.1, unstack.1)   │
├────────────────────────┤
│ (stack.1, stack.1)     │
├────────────────────────┤
│ (putdown.1, stack.2)   │
├────────────────────────┤
│ (putdown.1, stack.1)   │
╘════════════════════════╛


## AP matrix with vars

### Sort 1

╒═══════════╤════════════════════════╤══════════════════════╤═════════════╤═══════════════════════╤══════════════════════╤════════════════════════╕
│           │ unstack.1              │ stack.2              │   unstack.2 │ pickup.1              │ stack.1              │ putdown.1              │
╞═══════════╪════════════════════════╪══════════════════════╪═════════════╪═══════════════════════╪══════════════════════╪════════════════════════╡
│ unstack.1 │ nan                    │ (unstack.1, stack.2) │         nan │ (unstack.1, pickup.1) │ 1                    │ 1                      │
├───────────┼────────────────────────┼──────────────────────┼─────────────┼───────────────────────┼──────────────────────┼────────────────────────┤
│ stack.2   │ nan                    │ (stack.2, stack.2)   │           1 │ nan                   │ nan                  │ nan                    │
├───────────┼────────────────────────┼──────────────────────┼─────────────┼───────────────────────┼─────────────

## AP Constraints

╒══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ 1                                                                                                                                                        │
╞══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ AP_unstack.1~stack.2 >= [FO_0_B__5_unstack.1~_8_stack.2, FO_1_C__1_unstack.1~_6_stack.2, FO_1_B__3_unstack.1~_8_stack.2]                                 │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ AP_unstack.1~pickup.1 >= [FO_1_B__3_unstack.1~_5_pickup.1]                                                                                               │
├─────────────────────────────────────────────────────────

In [10]:
solver = pl.PULP_CBC_CMD(msg=True)
solution = POLOCM.polocm_step5(prob, solver,sort_AP_vars, True)

defaultdict(<class 'dict'>, {1: {(unstack.1, stack.2): AP_unstack.1~stack.2, (unstack.1, pickup.1): AP_unstack.1~pickup.1, (stack.2, stack.2): AP_stack.2~stack.2, (unstack.2, unstack.1): AP_unstack.2~unstack.1, (unstack.2, stack.1): AP_unstack.2~stack.1, (unstack.2, putdown.1): AP_unstack.2~putdown.1, (pickup.1, unstack.1): AP_pickup.1~unstack.1, (pickup.1, stack.2): AP_pickup.1~stack.2, (pickup.1, pickup.1): AP_pickup.1~pickup.1, (stack.1, unstack.1): AP_stack.1~unstack.1, (stack.1, stack.1): AP_stack.1~stack.1, (putdown.1, stack.2): AP_putdown.1~stack.2, (putdown.1, stack.1): AP_putdown.1~stack.1}})


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/uto/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9ce72c0e4490410d9f40a27f19f46835-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/9ce72c0e4490410d9f40a27f19f46835-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 906 COLUMNS
At line 3190 RHS
At line 4092 BOUNDS
At line 4300 ENDATA
Problem MODEL has 901 rows, 207 columns and 1856 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3 - 0.00 seconds
Cgl0002I 11 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 3 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory

In [11]:
FO, AP = POLOCM.polocm_step6(FO_matrix_with_vars, FO_vars_overall, sort_transition_matrix, sort_AP_vars, solution, debug=True)

## Solution FO matrix

### Trace 0

#### Obj A

╒══════════════╤════════════════╤════════════════╤═══════════════╤══════════════╕
│              │   [1]unstack.1 │   [2]putdown.1 │   [7]pickup.1 │   [8]stack.1 │
╞══════════════╪════════════════╪════════════════╪═══════════════╪══════════════╡
│ [1]unstack.1 │            nan │              1 │             0 │            0 │
├──────────────┼────────────────┼────────────────┼───────────────┼──────────────┤
│ [2]putdown.1 │            nan │            nan │             1 │            0 │
├──────────────┼────────────────┼────────────────┼───────────────┼──────────────┤
│ [7]pickup.1  │            nan │            nan │           nan │            1 │
├──────────────┼────────────────┼────────────────┼───────────────┼──────────────┤
│ [8]stack.1   │            nan │            nan │           nan │          nan │
╘══════════════╧════════════════╧════════════════╧═══════════════╧══════════════╛


#### Obj B

...

#### Obj A

╒════════════╤══════════════╕
│            │   [2]stack.2 │
╞════════════╪══════════════╡
│ [2]stack.2 │          nan │
╘════════════╧══════════════╛


## Solution AP matrix

### Sort 1

╒═══════════╤═════════════╤═══════════╤═════════════╤════════════╤═══════════╤═════════════╕
│           │   unstack.1 │   stack.2 │   unstack.2 │   pickup.1 │   stack.1 │   putdown.1 │
╞═══════════╪═════════════╪═══════════╪═════════════╪════════════╪═══════════╪═════════════╡
│ unstack.1 │         nan │         0 │         nan │          0 │         1 │           1 │
├───────────┼─────────────┼───────────┼─────────────┼────────────┼───────────┼─────────────┤
│ stack.2   │         nan │         0 │           1 │        nan │       nan │         nan │
├───────────┼─────────────┼───────────┼─────────────┼────────────┼───────────┼─────────────┤
│ unstack.2 │           1 │         1 │         nan │          1 │         0 │           0 │
├───────────┼─────────────┼───────────┼─────────────┼────────────┼───────────┼─────────────┤
│ pickup.1  │           0 │         0 │         nan │          0 │         1 │           1 │
├───────────┼─────────────┼───────────┼─────────────┼────────────┼────